# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 27 2023 8:30AM,256039,10,0086333238,ISDIN Corporation,Executing
1,Jan 27 2023 8:30AM,256040,10,0086333240,ISDIN Corporation,Released
2,Jan 27 2023 8:30AM,256039,10,0086333239,ISDIN Corporation,Executing
3,Jan 27 2023 8:30AM,256039,10,0086333237,ISDIN Corporation,Executing
4,Jan 27 2023 8:30AM,256039,10,0086333236,ISDIN Corporation,Executing
5,Jan 27 2023 8:30AM,256039,10,0086333370,ISDIN Corporation,Executing
6,Jan 27 2023 8:30AM,256039,10,0086333371,ISDIN Corporation,Executing
7,Jan 27 2023 8:30AM,256040,10,0086333374,ISDIN Corporation,Released
8,Jan 27 2023 8:30AM,256039,10,0086333373,ISDIN Corporation,Executing
9,Jan 27 2023 8:30AM,256039,10,0086333372,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,256039,Executing,7
20,256039,Released,12
21,256040,Released,5
22,256042,Released,1
23,256044,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256038,NaN,NaN,1.0
256039,NaN,7.0,12.0
256040,NaN,NaN,5.0
256042,NaN,NaN,1.0
256044,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255953,48.0,8.0,1.0
255969,0.0,1.0,0.0
255987,20.0,24.0,5.0
255997,0.0,0.0,1.0
256007,0.0,1.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
255953,48,8,1
255969,0,1,0
255987,20,24,5
255997,0,0,1
256007,0,1,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255953,48,8,1
1,255969,0,1,0
2,255987,20,24,5
3,255997,0,0,1
4,256007,0,1,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,255953,48,8,1
1,255969,,1,
2,255987,20,24,5
3,255997,,,1
4,256007,,1,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 27 2023 8:30AM,256039,10,ISDIN Corporation
1,Jan 27 2023 8:30AM,256040,10,ISDIN Corporation
10,Jan 27 2023 8:30AM,256035,10,ISDIN Corporation
30,Jan 26 2023 4:56PM,256044,22,FUJIFILM Diosynth Biotechnologies Texas LLC
31,Jan 26 2023 4:38PM,256042,19,ACG North America LLC
32,Jan 26 2023 3:43PM,256038,12,HealthCaps LLC
33,Jan 26 2023 3:25PM,256031,21,"NBTY Global, Inc."
34,Jan 26 2023 3:14PM,256030,16,TASA USA Inc.
35,Jan 26 2023 3:10PM,256029,16,TASA USA Inc.
36,Jan 26 2023 3:05PM,256028,16,TASA USA Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 27 2023 8:30AM,256039,10,ISDIN Corporation,,7,12
1,Jan 27 2023 8:30AM,256040,10,ISDIN Corporation,,,5
2,Jan 27 2023 8:30AM,256035,10,ISDIN Corporation,,4,2
3,Jan 26 2023 4:56PM,256044,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,,1
4,Jan 26 2023 4:38PM,256042,19,ACG North America LLC,,,1
5,Jan 26 2023 3:43PM,256038,12,HealthCaps LLC,,,1
6,Jan 26 2023 3:25PM,256031,21,"NBTY Global, Inc.",,,1
7,Jan 26 2023 3:14PM,256030,16,TASA USA Inc.,,,1
8,Jan 26 2023 3:10PM,256029,16,TASA USA Inc.,,,1
9,Jan 26 2023 3:05PM,256028,16,TASA USA Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 8:30AM,256039,10,ISDIN Corporation,12,7,
1,Jan 27 2023 8:30AM,256040,10,ISDIN Corporation,5,,
2,Jan 27 2023 8:30AM,256035,10,ISDIN Corporation,2,4,
3,Jan 26 2023 4:56PM,256044,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,,
4,Jan 26 2023 4:38PM,256042,19,ACG North America LLC,1,,
5,Jan 26 2023 3:43PM,256038,12,HealthCaps LLC,1,,
6,Jan 26 2023 3:25PM,256031,21,"NBTY Global, Inc.",1,,
7,Jan 26 2023 3:14PM,256030,16,TASA USA Inc.,1,,
8,Jan 26 2023 3:10PM,256029,16,TASA USA Inc.,1,,
9,Jan 26 2023 3:05PM,256028,16,TASA USA Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 8:30AM,256039,10,ISDIN Corporation,12,7,
1,Jan 27 2023 8:30AM,256040,10,ISDIN Corporation,5,,
2,Jan 27 2023 8:30AM,256035,10,ISDIN Corporation,2,4,
3,Jan 26 2023 4:56PM,256044,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,,
4,Jan 26 2023 4:38PM,256042,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 8:30AM,256039,10,ISDIN Corporation,12.0,7.0,NaN
1,Jan 27 2023 8:30AM,256040,10,ISDIN Corporation,5.0,NaN,NaN
2,Jan 27 2023 8:30AM,256035,10,ISDIN Corporation,2.0,4.0,NaN
3,Jan 26 2023 4:56PM,256044,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,NaN,NaN
4,Jan 26 2023 4:38PM,256042,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 8:30AM,256039,10,ISDIN Corporation,12.0,7.0,0.0
1,Jan 27 2023 8:30AM,256040,10,ISDIN Corporation,5.0,0.0,0.0
2,Jan 27 2023 8:30AM,256035,10,ISDIN Corporation,2.0,4.0,0.0
3,Jan 26 2023 4:56PM,256044,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,0.0,0.0
4,Jan 26 2023 4:38PM,256042,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,768114,19.0,11.0,0.0
12,256038,1.0,0.0,0.0
15,768005,11.0,25.0,20.0
16,768087,3.0,0.0,0.0
19,1024019,4.0,8.0,48.0
21,512000,1.0,1.0,0.0
22,256044,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,768114,19.0,11.0,0.0
1,12,256038,1.0,0.0,0.0
2,15,768005,11.0,25.0,20.0
3,16,768087,3.0,0.0,0.0
4,19,1024019,4.0,8.0,48.0
5,21,512000,1.0,1.0,0.0
6,22,256044,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,19.0,11.0,0.0
1,12,1.0,0.0,0.0
2,15,11.0,25.0,20.0
3,16,3.0,0.0,0.0
4,19,4.0,8.0,48.0
5,21,1.0,1.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,19.0
1,12,Released,1.0
2,15,Released,11.0
3,16,Released,3.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,0.0,0.0,20.0,0.0,48.0,0.0,0.0
Executing,11.0,0.0,25.0,0.0,8.0,1.0,0.0
Released,19.0,1.0,11.0,3.0,4.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,20.0,0.0,48.0,0.0,0.0
1,Executing,11.0,0.0,25.0,0.0,8.0,1.0,0.0
2,Released,19.0,1.0,11.0,3.0,4.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,20.0,0.0,48.0,0.0,0.0
1,Executing,11.0,0.0,25.0,0.0,8.0,1.0,0.0
2,Released,19.0,1.0,11.0,3.0,4.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()